In [1]:
import sf_quant as sf
import polars as pl
import datetime as dt

In [2]:
start = dt.date(2024, 1, 1)
end = dt.date(2024, 12, 31)

columns = [
    'date',
    'barrid',
    'ticker',
    'price',
    'return'
]

df = sf.data.assets.load(
    start=start,
    end=end,
    in_universe=True,
    columns=columns
)

df

date,barrid,ticker,price,return
date,str,str,f64,f64
2024-01-02,"""USA06Z1""","""MDXG""",7.87,-10.2623
2024-01-03,"""USA06Z1""","""MDXG""",7.775,-1.2071
2024-01-04,"""USA06Z1""","""MDXG""",7.76,-0.1929
2024-01-05,"""USA06Z1""","""MDXG""",7.8,0.5155
2024-01-08,"""USA06Z1""","""MDXG""",8.22,5.3846
…,…,…,…,…
2024-12-24,"""USBQOR1""","""ECG""",70.58,2.5872
2024-12-26,"""USBQOR1""","""ECG""",73.61,4.293
2024-12-27,"""USBQOR1""","""ECG""",69.85,-5.108
